In [27]:
from pymongo import MongoClient
from datetime import datetime

# Conexão com o MongoDB

In [3]:
MONGO_URI = "mongodb://0.0.0.0:27017"
client = MongoClient(MONGO_URI)

# Seleção do banco de dados e coleção

In [4]:
db = client["workshop_db"]
collection = db["workshop_collection"]

# Inserindo dados

In [28]:
documents = [
    {
        "name": "Alice",
        "age": 25,
        "city": "São Paulo",
        "hobbies": ["reading", "traveling"],
        "purchases": [
            {"item": "book", "price": 30, "date": datetime(2024, 12, 10)},
            {"item": "laptop", "price": 5000, "date": datetime(2024, 11, 5)},
        ],
    },
    {
        "name": "Bob",
        "age": 30,
        "city": "Rio de Janeiro",
        "hobbies": ["sports", "music"],
        "purchases": [
            {"item": "guitar", "price": 1500, "date": datetime(2024, 10, 15)},
            {"item": "headphones", "price": 200, "date": datetime(2024, 12, 1)},
        ],
    },
    {
        "name": "Charlie",
        "age": 35,
        "city": "Belo Horizonte",
        "hobbies": ["photography", "traveling"],
        "purchases": [
            {"item": "camera", "price": 3500, "date": datetime(2024, 8, 20)},
        ],
    },
    {
        "name": "Diana",
        "age": 28,
        "city": "Curitiba",
        "hobbies": ["cooking", "dancing"],
        "purchases": [
            {"item": "mixer", "price": 300, "date": datetime(2024, 9, 10)},
            {"item": "shoes", "price": 150, "date": datetime(2024, 10, 1)},
        ],
    },
    {
        "name": "Eve",
        "age": 22,
        "city": "Florianópolis",
        "hobbies": ["surfing", "yoga"],
        "purchases": [
            {"item": "surfboard", "price": 2500, "date": datetime(2024, 11, 18)},
            {"item": "yoga mat", "price": 100, "date": datetime(2024, 9, 5)},
        ],
    },
    {
        "name": "Frank",
        "age": 40,
        "city": "Porto Alegre",
        "hobbies": ["gardening", "cooking"],
        "purchases": [
            {"item": "grill", "price": 800, "date": datetime(2024, 6, 15)},
            {"item": "seeds", "price": 50, "date": datetime(2024, 7, 12)},
        ],
    },
    {
        "name": "Grace",
        "age": 27,
        "city": "Fortaleza",
        "hobbies": ["running", "reading"],
        "purchases": [
            {"item": "running shoes", "price": 300, "date": datetime(2024, 5, 10)},
            {"item": "e-book reader", "price": 700, "date": datetime(2024, 8, 25)},
        ],
    },
    {
        "name": "Henry",
        "age": 33,
        "city": "Brasília",
        "hobbies": ["cycling", "photography"],
        "purchases": [
            {"item": "bicycle", "price": 1200, "date": datetime(2024, 4, 8)},
            {"item": "tripod", "price": 150, "date": datetime(2024, 10, 12)},
        ],
    },
    {
        "name": "Ivy",
        "age": 29,
        "city": "Salvador",
        "hobbies": ["painting", "dancing"],
        "purchases": [
            {"item": "paint set", "price": 200, "date": datetime(2024, 3, 21)},
            {"item": "dance shoes", "price": 120, "date": datetime(2024, 9, 30)},
        ],
    },
    {
        "name": "Jack",
        "age": 36,
        "city": "Manaus",
        "hobbies": ["fishing", "hiking"],
        "purchases": [
            {"item": "fishing rod", "price": 400, "date": datetime(2024, 7, 1)},
            {"item": "backpack", "price": 350, "date": datetime(2024, 2, 14)},
        ],
    },
    {
        "name": "Karen",
        "age": 31,
        "city": "Recife",
        "hobbies": ["writing", "music"],
        "purchases": [
            {"item": "notebook", "price": 200, "date": datetime(2024, 1, 20)},
            {"item": "keyboard", "price": 600, "date": datetime(2024, 6, 7)},
        ],
    },
    {
        "name": "Leo",
        "age": 38,
        "city": "Belém",
        "hobbies": ["gaming", "coding"],
        "purchases": [
            {"item": "gaming chair", "price": 1200, "date": datetime(2024, 8, 5)},
            {"item": "monitor", "price": 2500, "date": datetime(2024, 12, 22)},
        ],
    },
]

result = collection.insert_many(documents)

In [29]:
print(f"Documentos inseridos: {result.inserted_ids}")

Documentos inseridos: [ObjectId('6762d87f2b477157b34639c0'), ObjectId('6762d87f2b477157b34639c1'), ObjectId('6762d87f2b477157b34639c2'), ObjectId('6762d87f2b477157b34639c3'), ObjectId('6762d87f2b477157b34639c4'), ObjectId('6762d87f2b477157b34639c5'), ObjectId('6762d87f2b477157b34639c6'), ObjectId('6762d87f2b477157b34639c7'), ObjectId('6762d87f2b477157b34639c8'), ObjectId('6762d87f2b477157b34639c9'), ObjectId('6762d87f2b477157b34639ca'), ObjectId('6762d87f2b477157b34639cb')]


# Consulta campos aninhados e arrays.

In [32]:
print("\nUsuários com hobbies que incluem 'traveling':")
for doc in collection.find({"hobbies": "traveling"}):
    print(doc)


Usuários com hobbies que incluem 'traveling':
{'_id': ObjectId('6762d87f2b477157b34639c0'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c2'), 'name': 'Charlie', 'age': 35, 'city': 'Belo Horizonte', 'hobbies': ['photography', 'traveling'], 'purchases': [{'item': 'camera', 'price': 3500, 'date': datetime.datetime(2024, 8, 20, 0, 0)}]}
{'_id': ObjectId('6762d95c83e4282d74afda5f'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}


In [30]:
consulta = {"name": "Alice"}

In [35]:
collection.find(consulta)[0]

{'_id': ObjectId('6762d87f2b477157b34639c0'),
 'name': 'Alice',
 'age': 25,
 'city': 'São Paulo',
 'hobbies': ['reading', 'traveling'],
 'purchases': [{'item': 'book',
   'price': 30,
   'date': datetime.datetime(2024, 12, 10, 0, 0)},
  {'item': 'laptop',
   'price': 5000,
   'date': datetime.datetime(2024, 11, 5, 0, 0)}]}

In [31]:
for doc in collection.find(consulta):
    print(doc)

{'_id': ObjectId('6762d87f2b477157b34639c0'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}


# Exemplo avançado de agregação.

In [36]:
pipeline = [
    {"$unwind": "$purchases"},  # Expande o array de compras
    {"$group": {
        "_id": "$name",
        "total_spent": {"$sum": "$purchases.price"},
        "average_spent": {"$avg": "$purchases.price"},
        "max_spent": {"$max": "$purchases.price"},
    }},
    {"$sort": {"total_spent": -1}},  # Ordena pelo total gasto (decrescente)
]

print("\nAgregação avançada (gastos totais por usuário):")
for result in collection.aggregate(pipeline):
    print(result)


Agregação avançada (gastos totais por usuário):
{'_id': 'Alice', 'total_spent': 10060, 'average_spent': 2515.0, 'max_spent': 5000}
{'_id': 'Leo', 'total_spent': 3700, 'average_spent': 1850.0, 'max_spent': 2500}
{'_id': 'Charlie', 'total_spent': 3500, 'average_spent': 3500.0, 'max_spent': 3500}
{'_id': 'Eve', 'total_spent': 2600, 'average_spent': 1300.0, 'max_spent': 2500}
{'_id': 'Bob', 'total_spent': 1700, 'average_spent': 850.0, 'max_spent': 1500}
{'_id': 'Henry', 'total_spent': 1350, 'average_spent': 675.0, 'max_spent': 1200}
{'_id': 'Grace', 'total_spent': 1000, 'average_spent': 500.0, 'max_spent': 700}
{'_id': 'Frank', 'total_spent': 850, 'average_spent': 425.0, 'max_spent': 800}
{'_id': 'Karen', 'total_spent': 800, 'average_spent': 400.0, 'max_spent': 600}
{'_id': 'Jack', 'total_spent': 750, 'average_spent': 375.0, 'max_spent': 400}
{'_id': 'Diana', 'total_spent': 450, 'average_spent': 225.0, 'max_spent': 300}
{'_id': 'Ivy', 'total_spent': 320, 'average_spent': 160.0, 'max_spent

# Consultas específicas com arrays.

In [38]:
print("\nUsuários com mais de uma compra:")
for doc in collection.find({"$expr": {"$gte": [{"$size": "$purchases"}, 2]}}):
    print(doc)


Usuários com mais de uma compra:
{'_id': ObjectId('6762d87f2b477157b34639c0'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c1'), 'name': 'Bob', 'age': 30, 'city': 'Rio de Janeiro', 'hobbies': ['sports', 'music'], 'purchases': [{'item': 'guitar', 'price': 1500, 'date': datetime.datetime(2024, 10, 15, 0, 0)}, {'item': 'headphones', 'price': 200, 'date': datetime.datetime(2024, 12, 1, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c3'), 'name': 'Diana', 'age': 28, 'city': 'Curitiba', 'hobbies': ['cooking', 'dancing'], 'purchases': [{'item': 'mixer', 'price': 300, 'date': datetime.datetime(2024, 9, 10, 0, 0)}, {'item': 'shoes', 'price': 150, 'date': datetime.datetime(2024, 10, 1, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c4'), '

In [39]:
print("\nUsuários que compraram algo em Outubro de 2024:")
for doc in collection.find({"purchases.date": {
    "$gte": datetime(2024, 10, 1),
    "$lt": datetime(2024, 11, 1),
}}):
    print(doc)


Usuários que compraram algo em Outubro de 2024:
{'_id': ObjectId('6762d87f2b477157b34639c1'), 'name': 'Bob', 'age': 30, 'city': 'Rio de Janeiro', 'hobbies': ['sports', 'music'], 'purchases': [{'item': 'guitar', 'price': 1500, 'date': datetime.datetime(2024, 10, 15, 0, 0)}, {'item': 'headphones', 'price': 200, 'date': datetime.datetime(2024, 12, 1, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c3'), 'name': 'Diana', 'age': 28, 'city': 'Curitiba', 'hobbies': ['cooking', 'dancing'], 'purchases': [{'item': 'mixer', 'price': 300, 'date': datetime.datetime(2024, 9, 10, 0, 0)}, {'item': 'shoes', 'price': 150, 'date': datetime.datetime(2024, 10, 1, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c4'), 'name': 'Eve', 'age': 22, 'city': 'Florianópolis', 'hobbies': ['surfing', 'yoga'], 'purchases': [{'item': 'surfboard', 'price': 2500, 'date': datetime.datetime(2024, 11, 18, 0, 0)}, {'item': 'yoga mat', 'price': 100, 'date': datetime.datetime(2024, 9, 5, 0, 0)}]}
{'_id': ObjectId('6762d87f2b

In [41]:
for doc in collection.find({
    "city": {"$in": ["São Paulo", "Rio de Janeiro", "Curitiba"]}
}):
    print(doc)

{'_id': ObjectId('6762d87f2b477157b34639c0'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c1'), 'name': 'Bob', 'age': 30, 'city': 'Rio de Janeiro', 'hobbies': ['sports', 'music'], 'purchases': [{'item': 'guitar', 'price': 1500, 'date': datetime.datetime(2024, 10, 15, 0, 0)}, {'item': 'headphones', 'price': 200, 'date': datetime.datetime(2024, 12, 1, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c3'), 'name': 'Diana', 'age': 28, 'city': 'Curitiba', 'hobbies': ['cooking', 'dancing'], 'purchases': [{'item': 'mixer', 'price': 300, 'date': datetime.datetime(2024, 9, 10, 0, 0)}, {'item': 'shoes', 'price': 150, 'date': datetime.datetime(2024, 10, 1, 0, 0)}]}
{'_id': ObjectId('6762d95c83e4282d74afda5f'), 'name': 'Alice', 'age': 25, 'city':

In [43]:
for doc in collection.find({
    "city": {"$nin": ["São Paulo", "Rio de Janeiro"]}
}):
    print(doc)

{'_id': ObjectId('6762d87f2b477157b34639c2'), 'name': 'Charlie', 'age': 35, 'city': 'Belo Horizonte', 'hobbies': ['photography', 'traveling'], 'purchases': [{'item': 'camera', 'price': 3500, 'date': datetime.datetime(2024, 8, 20, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c3'), 'name': 'Diana', 'age': 28, 'city': 'Curitiba', 'hobbies': ['cooking', 'dancing'], 'purchases': [{'item': 'mixer', 'price': 300, 'date': datetime.datetime(2024, 9, 10, 0, 0)}, {'item': 'shoes', 'price': 150, 'date': datetime.datetime(2024, 10, 1, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c4'), 'name': 'Eve', 'age': 22, 'city': 'Florianópolis', 'hobbies': ['surfing', 'yoga'], 'purchases': [{'item': 'surfboard', 'price': 2500, 'date': datetime.datetime(2024, 11, 18, 0, 0)}, {'item': 'yoga mat', 'price': 100, 'date': datetime.datetime(2024, 9, 5, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c5'), 'name': 'Frank', 'age': 40, 'city': 'Porto Alegre', 'hobbies': ['gardening', 'cooking'], 'purchases': [

In [45]:
for doc in collection.find({
    "$and": [
        {"city": {"$in": ["São Paulo", "Curitiba"]}},
        {"age": {"$gte": 25}}
    ]
}):
    print(doc)

{'_id': ObjectId('6762d87f2b477157b34639c0'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c3'), 'name': 'Diana', 'age': 28, 'city': 'Curitiba', 'hobbies': ['cooking', 'dancing'], 'purchases': [{'item': 'mixer', 'price': 300, 'date': datetime.datetime(2024, 9, 10, 0, 0)}, {'item': 'shoes', 'price': 150, 'date': datetime.datetime(2024, 10, 1, 0, 0)}]}
{'_id': ObjectId('6762d95c83e4282d74afda5f'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}


In [46]:
for doc in collection.find({
    "$or": [
        {"age": {"$lt": 30}},
        {"hobbies": "traveling"}
    ],
    "city": {"$nin": ["Rio de Janeiro"]}
}):
    print(doc)

{'_id': ObjectId('6762d87f2b477157b34639c0'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c2'), 'name': 'Charlie', 'age': 35, 'city': 'Belo Horizonte', 'hobbies': ['photography', 'traveling'], 'purchases': [{'item': 'camera', 'price': 3500, 'date': datetime.datetime(2024, 8, 20, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c3'), 'name': 'Diana', 'age': 28, 'city': 'Curitiba', 'hobbies': ['cooking', 'dancing'], 'purchases': [{'item': 'mixer', 'price': 300, 'date': datetime.datetime(2024, 9, 10, 0, 0)}, {'item': 'shoes', 'price': 150, 'date': datetime.datetime(2024, 10, 1, 0, 0)}]}
{'_id': ObjectId('6762d87f2b477157b34639c4'), 'name': 'Eve', 'age': 22, 'city': 'Florianópolis', 'hobbies': ['surfing', 'yoga'], 'purchases': [{'item': '

# Atualizações em campos específicos e arrays.

In [13]:
result = collection.update(
        {"name": "Alice"},
        {"$push": {"purchases": {"item": "phone", "price": 2000, "date": datetime(2024, 12, 15)}}}
    )
print(f"\nDocumentos atualizados: {result.modified_count}")


Documentos atualizados: 1


In [14]:
for doc in collection.find({"name": "Alice"}):
    print(doc)

{'_id': ObjectId('6762c7d03da68654183eaf7c'), 'name': 'Alice', 'age': 18, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}, {'item': 'phone', 'price': 2000, 'date': datetime.datetime(2024, 12, 15, 0, 0)}, {'item': 'phone', 'price': 2000, 'date': datetime.datetime(2024, 12, 15, 0, 0)}]}
{'_id': ObjectId('6762c97b19eadc1e843781aa'), 'name': 'Alice', 'age': 18, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'item': 'laptop', 'price': 5000, 'date': datetime.datetime(2024, 11, 5, 0, 0)}]}
{'_id': ObjectId('6762ce2d2b477157b34639a8'), 'name': 'Alice', 'age': 25, 'city': 'São Paulo', 'hobbies': ['reading', 'traveling'], 'purchases': [{'item': 'book', 'price': 30, 'date': datetime.datetime(2024, 12, 10, 0, 0)}, {'

# Exclusão baseada em critérios complexos.

In [15]:
result = collection.delete_many({"purchases.price": {"$gt": 3000}})
print(f"\nDocumentos excluídos: {result.deleted_count}")


Documentos excluídos: 5
